#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/manjunathdp/fullDataset.git
    

In [ ]:
!ls


In [ ]:
!ls


In [ ]:
# import shutil
# shutil.make_archive(OUTPUT_NAME, 'zip', DIRECTORY_TO_ZIP)
# import shutil
# shutil.make_archive('/kaggle/working/yolov5', 'zip', '/kaggle/working/')

In [ ]:
%cd 

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat /kaggle/working/yolov5/fullDataset/data.yaml

In [ ]:
# define number of classes based on YAML
import yaml
with open("/kaggle/working/yolov5/fullDataset/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
num_classes

In [ ]:
!ls /kaggle/working/yolov5/models

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /kaggle/working/yolov5/models/yolov5l.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /kaggle/working/yolov5/models/yolov5l.yaml
# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]


In [ ]:
# %%writetemplate /kaggle/working/yolov5/models/yolov5l.yaml

# # parameters
# nc: {num_classes}  # number of classes
# depth_multiple: 0.33  # model depth multiple
# width_multiple: 0.50  # layer channel multiple

# # anchors
# anchors:
#   - [10,13, 16,30, 33,23]  # P3/8
#   - [30,61, 62,45, 59,119]  # P4/16
#   - [116,90, 156,198, 373,326]  # P5/32

# # YOLOv5 backbone
# backbone:
#   # [from, number, module, args]
#   [[-1, 1, Focus, [64, 3]],  # 0-P1/2
#    [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
#    [-1, 3, BottleneckCSP, [128]],
#    [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
#    [-1, 9, BottleneckCSP, [256]],
#    [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
#    [-1, 9, BottleneckCSP, [512]],
#    [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
#    [-1, 1, SPP, [1024, [5, 9, 13]]],
#    [-1, 3, BottleneckCSP, [1024, False]],  # 9
#   ]

# # YOLOv5 head
# head:
#   [[-1, 1, Conv, [512, 1, 1]],
#    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
#    [[-1, 6], 1, Concat, [1]],  # cat backbone P4
#    [-1, 3, BottleneckCSP, [512, False]],  # 13

#    [-1, 1, Conv, [256, 1, 1]],
#    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
#    [[-1, 4], 1, Concat, [1]],  # cat backbone P3
#    [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

#    [-1, 1, Conv, [256, 3, 2]],
#    [[-1, 14], 1, Concat, [1]],  # cat head P4
#    [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

#    [-1, 1, Conv, [512, 3, 2]],
#    [[-1, 10], 1, Concat, [1]],  # cat head P5
#    [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

#    [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
#   ]

# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
!pwd

In [ ]:
%cd ..

In [ ]:
%%time

In [ ]:
%cd

## train yolov5s on custom data for 100 epochs
# time its performance
%time
#%cd /content/drive/MyDrive/yoloyolov5/
!python train.py --img 384 --batch 128 --epochs 100 --data /kaggle/input/bollworm/fullDataset/data.yaml --cfg /kaggle/working/yolov5/models/yolov5m.yaml --weights '' --name yolov5m_fullDataset_100  --cache

In [ ]:
!python train.py --img 384 --batch 64 --epochs 25 --data /kaggle/working/yolov5/fullDataset/data.yaml --cfg /kaggle/working/yolov5/models/yolov5l.yaml --weights '' --name yolov5l_fullDataset_25  

In [ ]:
!pwd
!ls


In [ ]:
#runs/train/yolov5l_fullDataset_25
#shutil.make_archive(OUTPUT_NAME, 'zip', DIRECTORY_TO_ZIP)
import shutil
shutil.make_archive('/kaggle/working/yolov5/runs/train/yolov5l_fullDataset_25', 'zip', '/kaggle/working/')

In [ ]:
%cd /kaggle/working/

In [ ]:
from IPython.display import FileLink 
FileLink(r'/kaggle/working/yolov5/runs/train/yolov5l_fullDataset_25/')

# Evaluate Custom YOLOv5 Detector Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs/train/yolov5l_fullDataset_25/runs/train

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
from IPython.display import Image
Image(filename='/content/drive/MyDrive/yolo/yolov5/runs/train/yolov5m_Y5_c/results.png', width=1000)  # view results.png

### Curious? Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
!pwd
!ls

In [ ]:
mydir_='/content/drive/MyDrive/yolo/'

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/drive/MyDrive/yolo/yolov5/runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename=mydir_+'yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/train/yolov5s_results/weights

In [ ]:

/content/drive/MyDrive/yolo/yolov5/worms-2/test/images

In [ ]:
!pwd

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/drive/MyDrive/yolo/yolov5/
!python detect1.py --weights runs/train/yolov5s_results/weights/best.pt --img 384 384 --conf 0.5 --source test --save-txt

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from numpy import asarray
dir='/content/drive/MyDrive/yolo/yolov5/runs/detect/exp12'
count=0
for img in os.listdir(dir):
  count+=1
  if(count>1):
    img_path=os.path.join(dir,img)
    img = cv2.imread(img_path)
    img =asarray(img)
  #img_array=cv2.imread(img_path)
  #Image(filename=img_path, width=900)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  #plt.imshow(img)
    plt.show()
  if(count==25):
    break
  #print(img_path)


In [ ]:
import os
import pandas as pd
ac=0
pc=0
abw=[]
pbw=[]
file=[]
tr=[]
DIRECTORY='/content/drive/MyDrive/yolo/yolov5/runs/detect/exp12/labels'
for txt in os.listdir(DIRECTORY):
    temp=txt.split("jpg")
    temp=temp[0].split('.')
    temp=temp[0]+'_'
    print(temp)
    #break
    ac=0
    pc=0
    txt_path=os.path.join(DIRECTORY,txt)
    with open(txt_path) as f:
        lines = f.readlines()

    for line in lines:
        tokens = line.split()
        if(tokens[0]=='0'):
            ac=ac+1
            #file.append(temp+'abw')
        if(tokens[0]=='1'):
            pc=pc+1

#     for token in tokens:
#         print (token)
    print("ac=",ac,"pc=",pc)
    abw.append(ac)
    file.append(temp+'abw')
    tr.append(ac)
    pbw.append(pc)
    file.append(temp+'pbw')
    tr.append(pc)
#print(abw,pbw,file)

In [ ]:
dftemp = pd.DataFrame({'image_id_worm':file,'number_of_worms':tr})
dftemp.head(8)

In [ ]:
dftemp.shape

In [ ]:
dftemp.to_csv('miniS1.csv',index=False)

In [ ]:
!ls

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/drive/MyDrive/yolo/yolov5/runs/detect/exp2/*.jpg'): #assuming JPG
    Image(filename=imageName, width=900)
    print(imageName)
    print("\n")

In [ ]:
print(imageName)

In [ ]:
!python export.py --weights yolov5s.pt --include engine --imgsz 384 --device 0  # export

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cp /content/yolov5/runs/train/yolov5s_results/weights/best.pt /content/gdrive/My\ Drive

## Congrats!

Hope you enjoyed this!

--Team [Roboflow](https://roboflow.ai)